# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
#     roads = Road.objects.filter(company=company).annotate(distance=Distance(point, 'path')).order_by('distance')
#     road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [13]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [14]:
!cat .env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [14]:
filename='Cópia de Dipositivos de Segurança-BRS-470 OK'

Escolhendo a aba da planilha do Excel

In [15]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Testando planilha carregada do Excel

In [16]:
print(ws['A1'].value)

km


Carregando os valores no script

In [17]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            for a in obj['img']:
                a.name = 'CSG'
#             obj['img'][0].name='Verso'
        except:
            pass

        
    values.append(obj)

Teste de valores importados da planilha

In [18]:
len(values),values[2]

(197,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.236402777777776,
  'Longitude_Inicial': -51.517425,
  'Latitude_Final': -29.23693888888889,
  'Longitude_Final': -51.517586111111115,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Crescente',
  'Classe': 'Dispositivos de Segurança',
  'Faixa': 'Não se aplica',
  'Rodovia': 'BRS-470',
  'Tipo': 'Defensa Metálica',
  'Extensão\n(m)': None,
  'Observações': None,
  'Estado de conservação': 'Bom',
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': None,
  'Foto_2': None,
  'Data Foto_2': None,
  'Tipo Foto_2': None,
  'Descrição Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4': None,
  'Tipo Foto_4': None,
  'Descrição Foto_4': None,
  'Foto_5': None,
  'Data

Definindo valores da importação

In [19]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Disp segurança
occurrence_type=OccurrenceType.objects.get(uuid='e5501855-f620-4664-88d4-ca9cb5370fda')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Dispositivos de Segurança - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>)

In [20]:
road=Road.objects.get(company=company,name__icontains="470")
items=list(road.marks.items())
first_km=items[0][1]['km']
last_km=items[-1][1]['km']
print(first_km,last_km)

220.5 233.39


In [22]:
print('Km inicial',' | ','km',' | ','km final')
for i,a in enumerate(tqdm(values)):
    road,_,_,km =point_to_km(float(str(a['Longitude_Inicial'])),float(str(a['Latitude_Inicial'])),road)
    if km >= first_km and km <= last_km:
        print(first_km,' | ',km,' | ',last_km)
        continue
    else:
        print(a['Longitude_Inicial'],a['Latitude_Inicial'],i+2)
# road,_,point,km =point_to_km(float(str(values[2]['Longitude_Inicial'])),float(str(values[2]['Latitude_Inicial'])),road)
# road,point,km

Km inicial  |  km  |  km final


  0%|          | 0/197 [00:00<?, ?it/s]

220.5  |  220.811  |  233.39
220.5  |  223.1  |  233.39
220.5  |  225.109  |  233.39
220.5  |  225.17  |  233.39
220.5  |  225.17  |  233.39
220.5  |  225.199  |  233.39
220.5  |  225.51  |  233.39
220.5  |  226.297  |  233.39
220.5  |  227.299  |  233.39
220.5  |  227.572  |  233.39
220.5  |  227.62  |  233.39
220.5  |  227.829  |  233.39
220.5  |  228.3  |  233.39
220.5  |  228.338  |  233.39
220.5  |  228.377  |  233.39
220.5  |  228.752  |  233.39
220.5  |  230.592  |  233.39
220.5  |  230.64  |  233.39
220.5  |  230.64  |  233.39
220.5  |  230.687  |  233.39
220.5  |  231.2  |  233.39
220.5  |  233.314  |  233.39
220.5  |  233.374  |  233.39
220.5  |  230.69  |  233.39
220.5  |  230.673  |  233.39
220.5  |  230.673  |  233.39
220.5  |  230.637  |  233.39
220.5  |  228.52  |  233.39
220.5  |  228.416  |  233.39
220.5  |  228.374  |  233.39
220.5  |  228.332  |  233.39
220.5  |  227.615  |  233.39
220.5  |  226.582  |  233.39
220.5  |  225.203  |  233.39
220.5  |  225.185  |  233.39

ValueError: could not convert string to float: 'None'

Criando itens de Inventário

In [ ]:
objects=[]


for index, a in enumerate(tqdm(values)):

    road,_,point,km =point_to_km(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']),road)
    _,_,_,end_km =point_to_km(float(a['Longitude_Final']),float(a['Latitude_Final']),road)
    if '470' not in road.name:
        print(f"Rodovia {road} - Km:{km} - Linha: {index+2}")
        continue

    try:

        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
                    end_km_manually_specified=True,
                    end_km=float(end_km),
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'mecanismoseguranca': get_value('Tipo', occurrence_type, a['Tipo'] if a['Tipo'] != 'Guarda-Corpo' else 'Guarda-corpo'),
                        'state': get_value('Estado de conservação', occurrence_type, a['Estado de conservação']),
                        'extension': a['Extensão\n(m)'],
                        'notes': a['Observações']
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/197 [00:00<?, ?it/s]

Salvando itens de Inventário no sistema e imprimindo os seriais

In [26]:
objects[70][0].__dict__

IndexError: list index out of range

In [26]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/274 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05904
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05905
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05906
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05907
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05908
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05909
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05910
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05911
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05977
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05978
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05979
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05980
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05981
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05982
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05983
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05984
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

CSG-Inv-2023.06049
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06050
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06051
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06052
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06053
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06054
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06055
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06056
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06057
Searching KM on direction...
Searching KM without direction...
Found KM on Ro

CSG-Inv-2023.06122
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06123
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06124
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06125
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06126
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06127
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06128
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06129
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.06130
Searching KM on direction...
Searching KM without direction...
Found KM on Ro

Anexando as imagens

In [ ]:
for reporting, images in tqdm(objects):
    for image in reversed(images):
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/274 [00:00<?, ?it/s]

In [22]:
reps = Reporting.objects.filter(company=company,occurrence_type=occurrence_type,created_at__gte='2023-5-10')
reps.count()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2023-05-10 00:00:00) while time zone support is active.
  RuntimeWarning)


76

In [23]:
for a in tqdm(reps):
    a.delete()

  0%|          | 0/76 [00:00<?, ?it/s]